# 00 - EU区数据库表探索

目标：连接OSAIO (`bi_center`) 和 Nooie (`nooie_bi_center`) 数据库，探索表结构和数据量。

**目标表**: `user`, `user_device`, `` `order` ``, `subscribe`

In [ ]:
import sys
sys.path.insert(0, '..')

from src.db.connector import DBConnector
from config.db_config import BRAND_DB_MAP, TARGET_TABLES
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## 1. 建立连接

In [ ]:
db = DBConnector(brand='osaio')
db.connect()
print(f"已连接: {db.database}")

## 2. OSAIO (bi_center) - 表探索

In [ ]:
# 查看所有表
tables = db.query_df("SHOW TABLES")
print(f"bi_center 共有 {len(tables)} 张表：")
tables

In [ ]:
# 目标表结构和数据量
for table in TARGET_TABLES:
    display_name = table.strip('`')
    print(f"\n{'='*60}")
    print(f"表: {display_name}")
    print('='*60)
    
    # 字段结构
    try:
        desc = db.query_df(f"DESCRIBE {table}")
        print(f"\n--- 字段结构 ({len(desc)} 列) ---")
        display(desc)
        
        # 数据量
        count = db.query_df(f"SELECT COUNT(*) as cnt FROM {table}")
        print(f"\n--- 数据量: {count['cnt'].iloc[0]:,} 行 ---")
        
        # 样本数据
        sample = db.query_df(f"SELECT * FROM {table} LIMIT 5")
        print(f"\n--- 样本数据 ---")
        display(sample)
    except Exception as e:
        print(f"查询失败: {e}")

## 3. Nooie (nooie_bi_center) - 表探索

In [ ]:
# 切换到Nooie数据库
db.switch_database('nooie')
print(f"已切换到: {db.database}")

In [ ]:
# 查看所有表
tables_nooie = db.query_df("SHOW TABLES")
print(f"nooie_bi_center 共有 {len(tables_nooie)} 张表：")
tables_nooie

In [ ]:
# 目标表结构和数据量
for table in TARGET_TABLES:
    display_name = table.strip('`')
    print(f"\n{'='*60}")
    print(f"表: {display_name}")
    print('='*60)
    
    try:
        desc = db.query_df(f"DESCRIBE {table}")
        print(f"\n--- 字段结构 ({len(desc)} 列) ---")
        display(desc)
        
        count = db.query_df(f"SELECT COUNT(*) as cnt FROM {table}")
        print(f"\n--- 数据量: {count['cnt'].iloc[0]:,} 行 ---")
        
        sample = db.query_df(f"SELECT * FROM {table} LIMIT 5")
        print(f"\n--- 样本数据 ---")
        display(sample)
    except Exception as e:
        print(f"查询失败: {e}")

## 4. 数据量汇总对比

In [ ]:
summary = []

for brand, database in BRAND_DB_MAP.items():
    db.switch_database(brand)
    for table in TARGET_TABLES:
        display_name = table.strip('`')
        try:
            count = db.query_df(f"SELECT COUNT(*) as cnt FROM {table}")
            summary.append({
                '品牌': brand.upper(),
                '数据库': database,
                '表': display_name,
                '行数': count['cnt'].iloc[0]
            })
        except Exception as e:
            summary.append({
                '品牌': brand.upper(),
                '数据库': database,
                '表': display_name,
                '行数': f'错误: {e}'
            })

summary_df = pd.DataFrame(summary)
print("=== 数据量汇总 ===")
summary_df

In [ ]:
# 关闭连接
db.close()
print("连接已关闭")